In [1]:
import pandas as pd
import minsearch_xtra as minsearch
import os
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
from openai import OpenAI

# Ingestion

In [2]:
pd.set_option('display.max_colwidth', None)  # Show all columns
df = pd.read_csv("../data/stoic_zen_document.csv")
df.insert(0,'id',df.index)

In [3]:
documents = df.to_dict(orient="records")
print("lenght of the documents:", len(documents))

index = minsearch.Index(
    text_fields=["category", "question", "answer"],
    keyword_fields=['id',"ideology"]
)

index.fit(documents)

lenght of the documents: 820


# Rag Flow

In [4]:
client = OpenAI()

def minsearch_search(query, ideology):
    boost = {'question': 3.0, 'category': 0.5}

    results = index.search(
        query=query,
        filter_dict={'ideology': ideology},
        boost_dict=boost,
        num_results=5
    )

    return results
    
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def build_prompt(query, search_results):
    prompt_template = """
    You're a philosophy teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION. Provide a real life quote proving your point,stating the author as well.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()

    entry_template = """
    category: {category}
    question: {question}
    answer: {answer}
    ideology: {ideology}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def rag(query,ideology):
    
    search_results = minsearch_search(query,ideology)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [5]:
query = "what are the cardinal basic virtues of stoicism?"
ideology = "stoicism"

answer = rag(query,ideology)
print(answer)

The cardinal basic virtues of Stoicism include courage, justice, and moderation. These virtues are not viewed in isolation but rather as unified aspects that derive from a comprehensive understanding of good and bad in various circumstances. 

As Seneca, a prominent Stoic philosopher, aptly stated, "A good character, for the best of men, consists in the pursuit of virtue." This highlights the integral role that virtues play in Stoic philosophy and the importance of embodying them.
